## Import Libraries

In [19]:
import sys
import os
import torch
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

## Configurations

In [20]:
SEQ_LEN = 7
INPUT_DIM = 6
HIDDEN_DIM = 64
NUM_STATES = 1
NUM_VARS = 6
MODEL_PATH = '/src/airflow/model/srvar_model_indices.pth'
CSV_PATH = '/src/data/future/SrVAR/indices_future_01_06_2025.csv'
MODEL_CODE_PATH = '/src/airflow'

## Import model

In [21]:
sys.path.append(os.path.abspath(MODEL_CODE_PATH))
from srvar_model import SrVAR

## Load model

In [22]:
model = SrVAR(INPUT_DIM, HIDDEN_DIM, NUM_STATES, NUM_VARS)
model.load_state_dict(torch.load(MODEL_PATH, map_location='cpu'))
model.eval()

SrVAR(
  (encoder): LSTMEncoder(
    (lstm): LSTM(6, 64, batch_first=True)
  )
  (state_regularized_unit): StateRegularizedUnit()
  (graph_generation): GraphGeneration(
    (g_a): Linear(in_features=64, out_features=36, bias=True)
    (g_d): Linear(in_features=64, out_features=6, bias=True)
  )
  (dynamic_var): DynamicVAR()
)

## Load Data

In [23]:
df = pd.read_csv(CSV_PATH)
df = df.sort_values('date')
features = df.drop(columns='date').columns
df_features = df[features]

## Scaling & Differencing data

In [24]:
scalers = {}
scaled_df = pd.DataFrame()
for col in features:
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scaled_df[col] = scaler.fit_transform(df[[col]]).flatten()
    scalers[col] = scaler

In [25]:
diff_df = scaled_df.diff().dropna().reset_index(drop=True)

In [27]:
input_seq = diff_df.iloc[-SEQ_LEN:].values
input_tensor = torch.tensor(input_seq, dtype=torch.float32).unsqueeze(0)

## Predict

In [28]:
with torch.no_grad():
    x_hat, alpha, entropy_reg, acyclic_penalty = model(input_tensor)

### Undo differencing and scaling data

In [29]:
last_scaled = scaled_df.iloc[-1].values
undiff_scaled = x_hat.squeeze(0).numpy() + last_scaled

In [30]:
unscaled = []
for i, col in enumerate(features):
    val = scalers[col].inverse_transform([[undiff_scaled[i]]])[0, 0]
    unscaled.append(val)

### Output

In [35]:
print("Predicted next real-world values (real):")
for name, val in zip(features, unscaled):
    print(f"{name}: {val:.2f}")

Predicted next real-world values (real):
gold: 3312.37
russell2000: 2065.78
dow_jones: 42268.27
msci_world: 3863.29
nasdaq100: 19111.76
s_p500: 5910.89
